**Code to extract data from Medical Expense and put it in a format suitable for IBNR**

* Output will have Paid Month, Service Month, and the total amount.
* There are two options.
    * The second cell is for individual months.
    * The third cell is for the entire folder (will take a few seconds more)

In [ ]:
#Import dependencies
import pandas as pd
import os
from FilePath import Data_Med_Path, Out_IBNR_Path

In [ ]:
#Run this code for individual file. Ask for input of file name
filemonth = input("What is the file month? ex: 201801 ")

#Specify and import data file as data frame: med
med_path = os.path.join(Data_Med_Path + filemonth + ".txt")
med = pd.read_csv(med_path,delimiter="|",dtype={'CLAIM_ID':float,'ICD_DIAG_22_POA':str,'ICD_DIAG_25_POA':str,'ICD_PROC_03':str,
                                                'ICD_PROC_04':str,'ICD_PROC_05':str,'ICD_PROC_06':str,'ICD_PROC_07':str,
                                                'ICD_PROC_08':str,'ICD_PROC_09':str,'ICD_PROC_10':str,'ICD_PROC_11':str,
                                                'ICD_PROC_12':str,'ICD_PROC_13':str,'ICD_PROC_14':str})
med.head()

In [ ]:
#Run this code for all files. Ask for how many files
index = ['201801', '201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812']
filemonth = int(input("How many files are there? "))

#Specify and import data file as data frame: med
for x in range(filemonth):
    med_path = os.path.join(Data_Med_Path + index[x] + ".txt")
    med_small = pd.read_csv(med_path,delimiter="|",dtype={'CLAIM_ID':float,'ICD_DIAG_22_POA':str,'ICD_DIAG_25_POA':str,
                                                          'ICD_PROC_03':str,'ICD_PROC_04':str,'ICD_PROC_05':str,'ICD_PROC_06':str,
                                                          'ICD_PROC_07':str,'ICD_PROC_08':str,'ICD_PROC_09':str,'ICD_PROC_10':str,
                                                          'ICD_PROC_11':str,'ICD_PROC_12':str,'ICD_PROC_13':str,'ICD_PROC_14':str})
    med = med.merge(med_small, how='outer')
med.head()

In [ ]:
#Store relevant columns into a smaller data frame called med_small
med_small = med[["CLAIM_ID", "FROM_DATE", "TO_DATE", "PAID_DATE", "BILL_PROV_NAME", "AMT_PAID", "POS", "REV_CODE", "PROC_CODE"]]
med_small.head(5)

In [ ]:
#Convert From_Date to Service_Month and Paid_Date to Paid_Month
med_small_date = pd.DataFrame(med_small)
med_small_date['FROM_YEAR'] = pd.DatetimeIndex(med_small['FROM_DATE']).year * 100
med_small_date['SERVICE_MONTH'] = pd.DatetimeIndex(med_small['FROM_DATE']).month + med_small_date['FROM_YEAR']
med_small_date['PAID_YEAR'] = pd.DatetimeIndex(med_small['PAID_DATE']).year * 100
med_small_date['PAID_MONTH'] = pd.DatetimeIndex(med_small['PAID_DATE']).month + med_small_date['PAID_YEAR']

#Drop year after calculation is finished, axis specified as 1 in order to call to series names
med_small_date = med_small_date.drop(["FROM_YEAR", "PAID_YEAR"], axis=1)

med_small_date.head()

In [ ]:
#Group By Month and Year and show total paid
by_month = med_small_date.groupby(["PAID_MONTH","SERVICE_MONTH"])
ibnr_frame = pd.DataFrame(by_month.sum()["AMT_PAID"])
ibnr_frame

In [ ]:
#Store the output dataframe.
if int(filemonth) < 20:
    ibnr_frame.to_excel(Out_IBNR_Path + "IBNR - All Up to " + index[filemonth-1] + ".xlsx")
else:
    ibnr_frame.to_excel(Out_IBNR_Path + "IBNR - " + str(filemonth) + ".xlsx")